<a href="https://colab.research.google.com/github/NavMohan-24/Machine-Learning-for-Semiconductor-Quantum-Devices/blob/main/module1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 1: Supervised learning for quantum dot configuration tuning
____

### Table of Contents

* [1. Charge stability diagram classification with dense Neural Networks](#part1)
    * [1.1. Import data and analyze the data shape](#Step_1)
    * [1.2. Prepare data](#Step_2)
    * [1.3. Setup the model](#Step_3)
    * [1.4. Compile the model](#Step_4)
    * [1.5. Train the model](#Step_5)
    * [1.6. Interpretation and analysis](#Step_6)


* [2. Exercise](#part2)


* [3. Charge stability diagram classification with Convolutional Neural Networks](#part3)
    * [3.1. Import data and analyze the data shape](#Step_1)
    * [3.2. Prepare data](#Step_2)
    * [3.3. Setup the model](#Step_3)
    * [3.4. Compile and train the model](#Step_4)
    * [3.5. Interpretation and analysis](#Step_5)


* [4. Exercise](#part4)

## 1. Charge stability diagram classification with dense Neural Networks <a class="anchor" id="part1"></a>
In this notebook we learn how to classify the charge stability diagram of single and double quantum dots with the help of Neural Networks (NNs).

The NN models that we are considering here are the fully connected neural networks and the Convolutional Neural Networks.
  
**Dataset**: The configurations are generated by our simulator on a 50x50 square lattice for both noisy and noiseless cases.

In [ ]:
!git clone https://gitlab.com/QMAI/mlqe_2023_edx.git

### 1.1. Import data and analyze the data shape <a class="anchor" id="Step_1"></a>

The folder `dataset` contains the data for noisy and noiseless charge stability diagrams.

In [ ]:
# Helper Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy as sp
from scipy.special import softmax
# Machine learning related libraries:
import torch
import torch.nn as nn            # base class used to develop all neural network models
import torch.nn.functional as F  # module of relu activation functions
import torch.optim as optim      # module of Adam optimizer
import glob
# from itertools import chain      # append two range() functions
from torch.utils.data import DataLoader # easy and organized data loading to the ML model
from torch.utils.data import Dataset #for nice loadable dataset creation

In [ ]:
####### Detect if running on the clusters  #######
# use CUDA:
torch.cuda.is_available()
print("Is cuda available?", torch.cuda.is_available())

# set a flag
device = torch.device("cuda:0")

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

In supervised learning we need to have labeled data, in the cell below we load the data and the respective labels.

In [ ]:
with open('mlqe_2023_edx/week3/dataset/csds.npy', 'rb') as f:
    data_noisy = np.load(f)

with open('mlqe_2023_edx/week3/dataset/csds_noiseless.npy', 'rb') as f:
    data = np.load(f)

with open('mlqe_2023_edx/week3/dataset/labels.npy', 'rb') as f:
    labels = np.load(f)

We have two sets of data, one is a noiseless (ideal) dataset and the other is a noisy dataset, similar to what we expect from an experiment. First we use the ideal dataset for training our system. You can also use the other set or change the size of the dataset, if you like.

In [ ]:
number_of_data = 200
data = data[:number_of_data]
labels = labels[:number_of_data]

Let'a visualize our data to see how each of phase diagram look like with their corresponding label.

In [ ]:

fig, ax = plt.subplots(1, 10, figsize = (20,10))
for index, d in enumerate(data[np.random.choice(len(data), size = 10)]):
    ax[index].imshow(data[index])
    ax[index].axis('off')
    ax[index].set_title(f'Label: {(labels[index])}')
plt.show()
plt.close()

### 1.2. Prepare data <a class="anchor" id="Step_2"></a>
Now that we know how our data set looks like, we need to make it readable for PyTorch

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.Tensor(data)
        self.labels = torch.Tensor(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_idx = self.data[idx]
        label = self.labels[idx].type(torch.LongTensor)
        return data_idx, label

Let's keep 20 percent of them for test set and 80 percent for train set and fix the batch size

In [ ]:
dataset = CustomDataset(data, labels)
trainset, testset = torch.utils.data.random_split(dataset, (int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)))

batch_size = 8

trainloader = DataLoader(trainset, batch_size = batch_size)
testloader = DataLoader(testset, batch_size = batch_size)


Then we can check the shape of the dataset.

In [ ]:
for X, y in trainloader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    print(y)
    break

### 1.3. Setup the model <a class="anchor" id="Step_3"></a>

Finally, we can set up the model!

One example of how to do that: input size is 50 x 50 = 2500, followed by one fully connected layer with _ReLU_ activation function and binary (1/0) output.

We define a class for the neural network. _torch_ requires this class to have a _.forward(x)_ method, and it is a convention to define the network during initialization of the class.

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(50*50,16),
            nn.ReLU(),
            nn.Linear(16,2),
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# here we initialize an instance of the class
model = NeuralNetwork().to(device)
# print the status of the model
# the print command is inherited from nn.Module in the definition of the network
print(model)

We can see the number of trainable parameters.

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {pytorch_total_params}")

### 1.4. Compile the model <a class="anchor" id="Step_4"></a>
Now that the model is defined, we need to train our model. But before doing so, there are a few details we need to specify.


1.   Loss function: we need to choose what function we want our model to minimise e.g. mean square, cross entropy.
2.   Optimisation method: How we want to update the weights e.g. stochastic gradient descent, ADAM.
3.   Metrics: some quantity we want to keep track of while we are training, e.g. value of the loss function or the accuracy of the model.


In [ ]:
# optimizing the model parameters
# to train the model we need a LOSS FUNCTION and an OPTIMIZER

loss_fn = nn.CrossEntropyLoss() # we use Cross Entropy as loss
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3) # The optimizer is Adam

In [ ]:
# define the train and test loss accumulation
train_loss_dnn = []
train_acc_dnn = []
test_loss_dnn = []
test_acc_dnn = []

In [ ]:
# define training function: make predicition on data set batch,
# backpropagate the error and adjust model parameters

def train(dataloader, model, loss_fn, optimizer, train_loss, train_acc):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    model.train()
    running_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        #compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # collect the accuracy:
        running_loss += loss.item()
        correct += (pred.argmax(1)==y).type(torch.float).sum().item()

        if batch % 25==0:
            loss,current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    running_loss /= num_batches
    correct /= size

    train_acc.append(correct)
    train_loss.append(running_loss)

#check performance against the test data set
def test(dataloader,model,loss_fn,test_loss, test_acc):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    running_loss, correct = 0, 0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            pred = model(X)
            running_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    running_loss /= num_batches
    correct /= size

    test_acc.append(correct)
    test_loss.append(running_loss)
    print(f"Test Error: \n Accuracy {(100*correct):>0.1f}%, Avg loss:{running_loss:>8f}\n")

### 1.5. Train the model <a class="anchor" id="Step_5"></a>

### Important note

In case you want to retrain your model for any purposes, you must **restart** you kernell first because you previous training data is already saved on your memory.

We can then use the defined functions to train the model for a certain number of epochs.

In [ ]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n -------------------")
    train(trainloader,model,loss_fn,optimizer, train_loss_dnn, train_acc_dnn)
    test(testloader,model,loss_fn, test_loss_dnn, test_acc_dnn)
print("Done!")

### 1.6. Interpretation and analysis <a class="anchor" id="Step_6"></a>


Here we define a function to plot the training and test accuracy/loss, which let us analyse the performance of the model.

In [ ]:
def create_acc_loss_graph(train_acc,train_loss, test_acc, test_loss):
    fig, axes = plt.subplots(ncols=2, nrows=1, dpi=300)
    fig.set_size_inches(9, 3)
    ax1, ax2 = axes[0], axes[1]

    ax1.plot(train_acc,'-o',label="train", markersize=4)
    ax1.plot(test_acc,'--+',label="test", markersize=4)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc=3)

    ax2.plot(train_loss,'-o',label="train", markersize=4)
    ax2.plot(test_loss,'--+',label=" test", markersize=4)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc=1)

    #plot parameters
    ax1.set_ylim(0, np.max([np.max(train_acc),np.max(test_acc)])+0.1)
    ax2.set_ylim(-0.1, np.max([np.max(train_loss),np.max(test_loss)])+0.1)
    ax1.grid(True, which='both',linewidth=0.1)
    ax2.grid(True, which='both',linewidth=0.1)
    plt.tight_layout()
    plt.show()

In [ ]:
create_acc_loss_graph(train_acc_dnn,train_loss_dnn, test_acc_dnn, test_loss_dnn)

## 2. Exercise <a class="anchor" id="part2"></a>

Now it is time to get your hands dirty and do some coding. Repeat the process above to train the model for noisy data. How the training changes?

### Important note

In case you want to retrain your model for any purposes, you must **restart** you kernell first because you previous training data is already saved on your memory.

## 3. Charge stability diagram Classification with Convolutional Neural Networks (CNN) <a class="anchor" id="part3"></a>


We have introduced here a new layer. Lets briefly understand what each layer does.

1.  **Convolutional**: This layer applies 32 kernels of size 2 by 2 over the input image. There are 2 paddings one can choose from 'valid' or 'same'. For our purpose, we need periodic boundary conditions and we thus use 'Valid', which means it does not add additional 'pixels' around the configuration.
```
torch.nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride=1, padding=1)
```




For more information about layer check out the PyTorch documentation: https://pytorch.org/docs/stable/nn


One final note on the input shape: While for the (first) dense layer we can define just about any shape, the input shape for the convolutional layer is necessarily N x M x C, where C is the number of channels. If the input available has only a single channel, i.e., its shape is N x M, an additional axis with dimension 1 needs to be added for it to work.

Animation: What convolutional layers do?

![Alt Text](https://miro.medium.com/max/789/0*jLoqqFsO-52KHTn9.gif)

The yellow matrix is called a kernel, and its size is one of the hyperparameters. It moves around the green (input) image with step defined by `stride` (here = 1), and how it behaves at the edges of the image is called `padding`. The resulting convolved image is an input to a next layer.

Note that the convolution is performed simultaneously for each channel of the input image, e.g. a color image has C=3 channels, RGB: Red, Green, and Blue. The filters are set to have odd size for practical purpose CxFxF, e.g, 3x3x3, 3x5x5. The output of this operation is one scalar value, an artificial neuron. An illustrative animation for the convolution layer is given in the course by Fei-Fei Li. More details: http://cs231n.github.io/convolutional-networks/#conv.

![Alt Text](https://miro.medium.com/max/1100/1*qtinjiZct2w7Dr4XoFixnA.gif)


### 3.2. Prepare data <a class="anchor" id="Step_2"></a>
Now let's prepare our data again, this time we choose only 100 data for training and 100 for test which is almost half of what we used in the dense neural network. Keep in mind that we changed the shape of the input data from [100,50,50] to [100,1,50,50] because CNN in the PyTorch mainly designed for image processing and images usually have 3 channels, RGB but here we only need 1 channel.

In [ ]:
number_of_data = 100

with open('mlqe_2023_edx/week3/dataset/csds.npy', 'rb') as f:
    data_noisy = np.load(f)

with open('mlqe_2023_edx/week3/dataset/csds_noiseless.npy', 'rb') as f:
    data = np.load(f)

with open('mlqe_2023_edx/week3/dataset/labels.npy', 'rb') as f:
    labels = np.load(f)

data_val = data[-number_of_data:]
data_train = data[:number_of_data]

labels_val = labels[-number_of_data:]
labels_train = labels[:number_of_data]

In [ ]:
dataset = CustomDataset(data_train, labels_train)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_val = CustomDataset(data_val, labels_val)
data_loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=True)



In [ ]:
for i, (x,y) in enumerate(data_loader):
    if i == 1: break
    print(x.shape, y.shape)

### 3.3. Setup the model <a class="anchor" id="Step_3"></a>

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding =1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.fc1 = nn.Linear(in_features=64 * 11 * 11, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=2)

    def forward(self, x):
        x = x.view(-1, 1, 50, 50)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 10

learning_rate = 0.001

### 3.4. Compile and train the model <a class="anchor" id="Step_4"></a>


Clearing the memory

In [ ]:
# Initialize the network
model = Net().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
total_step = len(data_loader)
train_ac = []
train_Loss = []
test_Loss = []
test_ac = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #if (i + 1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}")
    train_Loss.append(loss.item())
    # Calculate accuracy after each epoch
    model.eval()
    with torch.no_grad():
        input_data = torch.tensor(data_train).to(torch.float32).to(device)
        target_data = torch.tensor(labels_train).to(torch.int)
        print(input_data.shape)
        outputs = model(input_data)
        _, predicted = torch.max(outputs.data, 1)

    # Calculate accuracy
    accuracy = (predicted == target_data).sum().item() / len(target_data)
    train_ac.append(accuracy)
    print(f"Accuracy after Epoch {epoch+1}: {accuracy}")

    # Evaluation on test dataset
    model.eval()  # Set the model to evaluation mode
    running_test_loss = 0.0
    correct_test_predictions = 0

    for images, labels in data_loader_val:
        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(images)
            test_loss = criterion(outputs, labels)
            running_test_loss += test_loss.item()

            _, predicted = torch.max(outputs.data, 1)
            correct_test_predictions += (predicted == labels).sum().item()

    # Calculate accuracy and average loss for the test dataset
    test_accuracy = correct_test_predictions / len(data_val)
    test_ac.append(test_accuracy)
    test_average_loss = running_test_loss / len(data_loader_val)
    test_Loss.append(test_average_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Test Loss: {test_average_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")



### Important note

In case you want to retrain your model for any purposes, you must **restart** you kernell first because you previous training data is already saved on your memory.

### 1.5. Interpretation and analysis <a class="anchor" id="Step_5"></a>


In [ ]:
def create_acc_loss_graph2(train_loss, train_acc, test_loss,test_acc):
    fig, axes = plt.subplots(ncols=2, nrows=1, dpi=300)
    fig.set_size_inches(9, 3)
    ax1, ax2 = axes[0], axes[1]


    ax1.plot(train_acc,'-o',label="train", markersize=4)
    ax1.plot(test_acc,'--+',label="test", markersize=4)
    ax1.plot()
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend(loc=3)

    ax2.plot(train_loss,'-o',label="train", markersize=4)
    ax2.plot(test_loss,'--+',label="test", markersize=4)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend(loc=1)

    #plot parameters
    #ax1.set_ylim(0, np.max([np.max(train_acc),np.max(test_acc)])+0.1)
    #ax2.set_ylim(-0.1, np.max([np.max(train_loss),np.max(test_loss)])+0.1)
    ax1.grid(True, which='both',linewidth=0.1)
    ax2.grid(True, which='both',linewidth=0.1)
    plt.tight_layout()
    plt.show()

In [ ]:
create_acc_loss_graph2(train_Loss,train_ac,test_Loss,test_ac)

### 3.4. Exercise <a class="anchor" id="part4"></a>

Now it is time to get your hands dirty and do some coding. Repeat the process above to train the model for noisy data. How the training changes?

### Important note

In case you want to retrain your model for any purposes, you must **restart** you kernell first because you previous training data is already saved on your memory.